# IMPORT LIBARIES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("SampleSuperstore.csv")


## GET INFO ABOUT THE DATASET

In [ ]:
df.info()
df.describe()
print(df.columns)  # Lists all column names
df.dtypes

# CHANGE THE DATA TYPE OF COLUMNS

In [ ]:
df["Order Date"] = pd.to_datetime(df["Order Date"], format="%d/%m/%Y", errors='coerce')
df["Ship Date"] = pd.to_datetime(df["Ship Date"], format="%d/%m/%Y", errors='coerce')

# check the data types again
print(df.dtypes)


# CHECK MISSING AND DUPLICATE VALUES

In [ ]:
df.isnull().sum()
df.duplicated().sum()

In [ ]:
pd.to_datetime(df['Order Date'], format='%d/%m/%Y', errors='coerce').isnull().any()
#   .isnull().any()   :-  checks if there are any null values(NaT values) . If there are no null values, it returns False, and if there are any, it returns True.
#   errors = "coerce" :-   If any value cannot be converted, it will be replaced with NaT (Not a Time) 

# missing postal code

In [ ]:
missing_postal_codes = df[df["Postal Code"].isna()]
print(missing_postal_codes)

### Since all the missing postal code is from burlington vermont we will replace it with 0

In [ ]:
df.loc[(df["City"] == "Burlington") & (df["State"] == "Vermont") & df["Postal Code"].isna(), "Postal Code"] = 0
df["Postal Code"] = df["Postal Code"].astype(int)

## final check and cleaning

In [ ]:
# ensures each customer have unique customer id 

duplicates = df.groupby("Customer ID")["Customer Name"].nunique()
print(duplicates[duplicates > 1])

print(df[df["Sales"] < 0])   # Unusual negative sales?

df.info()

df.to_csv("SampleSuperstore.csv", index=False)

# -------------------------- VISUALIZATION ---------------------------

In [ ]:
# Sales by Category and Sub-Category
category_sales = df.groupby("Category")["Sales"].sum().sort_values(ascending=False)
subcategory_sales = df.groupby("Sub-Category")["Sales"].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(x=category_sales.index, y=category_sales.values, palette="viridis")
plt.title("Total Sales by Category")
plt.xlabel("Category")
plt.ylabel("Total Sales")
plt.grid(axis="y")
plt.show()
print(category_sales)


plt.figure(figsize=(12, 6))
sns.barplot(x=subcategory_sales.index, y=subcategory_sales.values, palette="coolwarm")
plt.title("Total Sales by Sub-Category")
plt.xlabel("Sub-Category")
plt.ylabel("Total Sales")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.show()

In [ ]:
region_sales = df.groupby("Region")["Sales"].sum().sort_values(ascending=False)
state_sales = df.groupby("State")["Sales"].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 5))
sns.barplot(x=region_sales.index, y=region_sales.values, palette="pastel")
plt.title("Total Sales by Region")
plt.xlabel("Region")
plt.ylabel("Total Sales")
plt.grid(axis="y")
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x=state_sales.index, y=state_sales.values, palette="magma")
plt.title("Top 10 States by Sales")
plt.xlabel("State")
plt.ylabel("Total Sales")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.show()

In [ ]:
df["Shipping Duration"] = (df["Ship Date"] - df["Order Date"]).dt.days
shipping_mode_duration = df.groupby("Ship Mode")["Shipping Duration"].mean().sort_values()

plt.figure(figsize=(10, 5))
sns.barplot(x=shipping_mode_duration.index, y=shipping_mode_duration.values, palette="Blues_r")
plt.title("Average Shipping Duration by Ship Mode")
plt.xlabel("Ship Mode")
plt.ylabel("Average Shipping Time (Days)")
plt.grid(axis="y")
plt.show()



In [ ]:
# Top Customers
top_customers = df.groupby("Customer Name")["Sales"].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_customers.values, y=top_customers.index, palette="viridis")
plt.title("Top 10 Customers by Total Sales")
plt.xlabel("Total Sales")
plt.ylabel("Customer Name")
plt.grid(axis="x")
plt.show()

In [ ]:
# Sales by Customer Segment
segment_sales = df.groupby("Segment")["Sales"].sum().sort_values(ascending=False)
plt.figure(figsize=(8, 5))
sns.barplot(x=segment_sales.index, y=segment_sales.values, palette="coolwarm")
plt.title("Total Sales by Customer Segment")
plt.xlabel("Customer Segment")
plt.ylabel("Total Sales")
plt.grid(axis="y")
plt.show()

In [ ]:
# Repeat Customer Analysis
customer_order_count = df.groupby("Customer Name").size().sort_values(ascending=False)
repeat_customers = customer_order_count[customer_order_count > 1]
plt.figure(figsize=(12, 6))
sns.histplot(repeat_customers, bins=30, kde=True, color="purple")
plt.title("Distribution of Repeat Customer Orders")
plt.xlabel("Number of Orders")
plt.ylabel("Number of Customers")
plt.grid(axis="y")
plt.show()

In [ ]:
# Sales Trend Over Time
monthly_sales = df.groupby(df["Order Date"].dt.to_period("M"))["Sales"].sum()
plt.figure(figsize=(12, 6))
monthly_sales.plot(marker="o", linestyle="-", color="b")
plt.title("Sales Trend Over Time")
plt.xlabel("Year-Month")
plt.ylabel("Total Sales")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
year = int(input("Enter the year: "))  

if year < 2015 or year > 2018:  
    print('No record is found')

else:
    df_year = df[df["Order Date"].dt.year == year]

    monthly_sales = df_year.groupby(df_year["Order Date"].dt.month)["Sales"].sum()

    # Ensure all months are represented
    monthly_sales = monthly_sales.reindex(range(1, 13), fill_value=0)

    # Map numeric months to names
    month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    monthly_sales.index = month_names

    # Plot
    plt.figure(figsize=(12, 6))
    monthly_sales.plot(marker="o", linestyle="-", color="b")
    plt.title(f"Monthly Sales for {year}")
    plt.xlabel("Month")
    plt.ylabel("Total Sales")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

    print(monthly_sales.to_frame())
